# Fish_EXP


## Hyperparameters

In [5]:


#torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import optim
#Libaries for calculation and processing
from einops import rearrange, repeat
import math
from math import sqrt
from math import ceil
import numpy as np
from sklearn.preprocessing import StandardScaler
#libaries for data importng, formating and handling 
import pandas as pd
#For analysis and plotting
import matplotlib.pyplot as plt
import seaborn as sns
#others
import os
import time
import json
import pickle
import warnings
warnings.filterwarnings('ignore')


from model.Crossformer import *
from data.Dataset import Dataset_MTS,Dataset_MTS_simplified
from exp.ExpFish import Expfish

In [11]:

#DATASET 
ROOT_PATH = "data/DatasetClusters/fishes/fish02/"
DATA_PATH = "s1/df_timeseries_red_clusters_s1.csv"
TRAIN_FLAG = "train"
VAL_FLAG = "val"
TEST_FLAG = "test"
SIZE=[50,50,50] #[seq_len, label_len, pred_len]
SCALE = True
SCALE_STATISTIC = True
DATA_SPLIT = [1, 0, 0.0]  # Train, Val, TEST
STRIDE = 1

#DATALOADER 
BATCH_SIZE = 10
SHUFFLE_FLAG = False
NUM_WORKSES = 0
DROP_LAST = False

#MODEL 
DATA_DIM = 2 # number of clusers
IN_LEN   = SIZE[0]
OUT_LEN  = SIZE[2]
SEG_LEN  = 10
WIN_SIZE = 1
FACTOR   = 2
D_MODEL  = 256 
D_FF     = 512
N_HEADS  = 1
E_LAYERS = 1
DROPOUT  = 0.2
BASELINE = False

#Device
DEVICE   = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#OPTIMIZER
LR = 0.001

#TRAINING 
NUM_EPOCHS = 2



## DataLoader

In [12]:
train_set = Dataset_MTS(root_path=ROOT_PATH,data_path=DATA_PATH,flag=TRAIN_FLAG,size=SIZE,scale=SCALE,scale_statistic=SCALE_STATISTIC,data_split=DATA_SPLIT,stride=STRIDE)


data_loader_train = DataLoader(
            train_set,
            batch_size=BATCH_SIZE,
            shuffle=SHUFFLE_FLAG,
            num_workers=NUM_WORKSES,
            drop_last=DROP_LAST
            )

## Model Init


In [13]:
model = Crossformer(
    data_dim=DATA_DIM,
    in_len=IN_LEN,
    out_len=OUT_LEN,
    seg_len=SEG_LEN,
    win_size=WIN_SIZE,
    factor=FACTOR,
    d_model=D_MODEL,
    d_ff=D_MODEL,
    n_heads=N_HEADS,
    e_layers=E_LAYERS,
    dropout=DROPOUT,
    baseline=False,
    device=DEVICE
    
).float()

## Loss-function and Optimizer

In [14]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

## Training

In [15]:
exp = Expfish(model=model,
               data_loader_train=data_loader_train,
               data_loader_test=None,
               data_loader_val=None,
               optimizer=optimizer,
               criterion=criterion,
               num_epochs=NUM_EPOCHS,
               device=DEVICE
               
               )

model = exp.train()


Epoch: 1 cost time: 6.283841848373413
Epoch: 1, Steps: 94 | Train Loss: 0.9975803 Vali Loss: 0.0000000
Epoch: 2 cost time: 5.832176446914673
Epoch: 2, Steps: 94 | Train Loss: 1.1206553 Vali Loss: 0.0000000
